In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as fun
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import numpy as np
import random
import math
import jieba #中文分词工具
import pandas as pd

model_cn = torch.load("C:\\Users\\29147\\source\\repos\\NLP初级知识整合\\NLP-summary\\cn_in_embed.pth")
model_cn.eval()
model_cn = model_cn.cpu()

model_en = torch.load("C:\\Users\\29147\\source\\repos\\NLP初级知识整合\\NLP-summary\\en_in_embed.pth")
model_en.eval()
model_en = model_en.cpu()


with open("news-commentary-v13.zh-en.cn","r",encoding='utf-8') as f:
    text_cn = f.read()

with open("news-commentary-v13.zh-en.en","r",encoding='utf-8') as f:
    text_en = f.read()

#   当不给split函数传递任何参数时，分隔符sep会采用任意形式的空白字符
text_en = text_en.split()   #英文单词直接用空格分就行
text_cn = jieba.lcut(text_cn,cut_all=False)   #用jieba的精确模式对中文进行分词

vocab_en = dict(Counter(text_en))
vocab_cn = dict(Counter(text_cn))

mappingCN_index2word = [word for word in vocab_cn.keys()]
mappingCN_word2index = {word:i for i,word in enumerate(mappingCN_index2word)}

mappingEN_index2word = [word for word in vocab_en.keys()]
mappingEN_word2index = {word:i for i,word in enumerate(mappingEN_index2word)}

In [59]:
def ShowNearest(word,word2index,index2word,embed_model):
    # 记录十个相似度最高的词的下标和相似度
    sim = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]   
    indx = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

    wordVec = embed_model(torch.tensor(word2index[word])).detach().numpy()
    targetVecs = [embed_model(torch.tensor(index)).detach().numpy() for index in range(len(index2word))]
    for target_index in range(len(targetVecs)):
        similarity = cosine_similarity([wordVec], [targetVecs[target_index]])#非得用2维数组比
        for sim_idx in range(len(sim)):
            #   
            if(similarity > sim[sim_idx]):
                sim[sim_idx] = similarity
                indx[sim_idx] = target_index
                break
            

    return [index2word[i] for i in indx]

In [64]:

#余弦相似度最高的10个词
#print(ShowNearest("中国",mappingCN_word2index,mappingCN_index2word,model_cn))
print(ShowNearest("virus",mappingEN_word2index,mappingEN_index2word,model_en))
print(ShowNearest("India",mappingEN_word2index,mappingEN_index2word,model_en))
print(ShowNearest("climate",mappingEN_word2index,mappingEN_index2word,model_en))
print(ShowNearest("economic",mappingEN_word2index,mappingEN_index2word,model_en))
print(ShowNearest("crisis",mappingEN_word2index,mappingEN_index2word,model_en))
print(ShowNearest("weapon",mappingEN_word2index,mappingEN_index2word,model_en))

['virus', 'gradual', 'China.', 'transformation', 'gone', 'clean', 'positions', 'liberalization', 'interest-rate', 'Rather,']
['India', 'In', 'like', 'For', 'where', 'Western', 'leading', 'American', 'into', 'within']
['climate', 'change.', 'comprehensive', 'key', 'problems', 'progress', 'strategy', 'critical', 'reform', 'building']
['economic', 'global', 'potential', 'significant', 'stability', 'China’s', 'further', 'creating', 'national', 'sustained']
['crisis', 'crisis.', 'economy', 'current', 'economy.', 'began', 'third', 'Greek', 'fact,', 'ongoing']
['weapon', 'cooperative', 'low-carbon', 'emissions.', 'capital,', 'rights,', 'security,', 'contribute', 'security.', 'supplies']


In [65]:
print(ShowNearest("病毒",mappingCN_word2index,mappingCN_index2word,model_cn))
print(ShowNearest("能源",mappingCN_word2index,mappingCN_index2word,model_cn))
print(ShowNearest("中国",mappingCN_word2index,mappingCN_index2word,model_cn))
print(ShowNearest("北京",mappingCN_word2index,mappingCN_index2word,model_cn))
print(ShowNearest("经济",mappingCN_word2index,mappingCN_index2word,model_cn))

['病毒', '感染', '疾病', '艾滋病', '预防', '群体', '药物', '普遍', '致命', '其实']
['能源', '生产', '行业', '效率', '绿色', '水', '清洁', '卫生', '用于', '土地']
['中国', '美国', '尽管', '已', '由于', '日本', '正', '印度', '部分', '转变']
['北京', '刚刚', '－', '纽约', '正如', '曾经', '香港', '期间', '著名', '回归']
['经济', '繁荣', '阻碍', '模式', '当前', '转变', '转型', '前所未有', '一体化', '缓慢']
